# Anomaly detection using Facebook Prophet:

**Medical background:**

In the last decades, the miniaturization of wearable sensors and development of data transmission technologies have allowed to collect medically relevant data called digital biomarkers. This data is revolutionising our modern Medicine by offering new perspectives to better understand the human physiology and the possibility to better identify, even predict disease progression.
Yet, the data collected by wearable sensors is, for technical reasons heterogeneous and cannot be directly translated into a meaningful clinical status. The relevance of proper data analysis is extremely critical as a wrong data analysis may lead to miss critical disease progression steps or lead to the wrong diagnostic. Therefore, the overall goal of this project is to integrate patients’ sensor data into one or several outcome measures which meaningfully recapitulate the clinical status of the patient.

[Stress](https://en.wikipedia.org/wiki/Stress_(biology) is a natural and physiological response to threat, challenge or physical and psychological barrier. In humans, the two major systems leading responding to stress are the autonomic nervous system and hypothalamic-pituitary-adrenal axis. The sympathetic nervous system, the stress-related part of the autonomic nervous system aims to distribute the energy to the most relevant body parts, to react to the stressor by fighting or escaping for instance. The hypothalamic-pituitary-adrenal axis regulates metabolic, psychological and immunological functions.
The adrenaline alters the following: motion rate, electrocardiogram (ECG), electrodermal activity (EDA), electromyogram (EMG), respiration and body temperature.

**Goal of the script:**

Here, we aim leverage the power of artificial intelligence to reach a medical insight. Specifically, we want to detect the stress-induced biological changes from the wearable device measure with the highest sensiticity.

**Motivations to use a forecasting method to detect activity:**

Previous works demonstrated the ability to related self-labeled stress status to sensor data acquired by wearable sensors.
Here, we try a different approach assuming that physiological rythms are altered by stress. We are investigating if a time series forecasting method coupled with anomaly detection provides a more sensitive methods to detect stress-related changes.

**Data format**

The reader may read the original source of data here:
- [UCI website](https://archive.ics.uci.edu/ml/datasets/WESAD+%28Wearable+Stress+and+Affect+Detection%29) (check the website shown below) to download the WESAD dataset 
- [wesad_readme file](wesad_readme.pdf) and [wesad poster](WESAD poster.pdf), both located together with the WESAD dataset

**Structure of the code**

    1 - Read the data
    2 - Data preparation: segmentation per task, quality control
    3 - Prediction of sensor data in the absence of change of stress status
    4 - Detection of anomaly in the sensor's data, indicating a change in the stress status
    4 - Cross-validation and boot-strapping assess the robustness of each candidate model and generate estimates of variability to facilitate model selection

**Credit**

- The data extraction part was modified from a script written by [aganjag](https://github.com/jaganjag) and is available [here](https://github.com/jaganjag/stress_affect_detection/blob/master/prototype.ipynb)
- The future implementation of Prophet for Time Series Forecasting was based on this [tutorial](https://medium.com/analytics-vidhya/time-series-forecast-anomaly-detection-with-facebook-prophet-558136be4b8d) written by Paul Lo. It makes use of the open-source project [Prophet](https://facebook.github.io/prophet/), a forecasting procedure implemented in R and Python, based on the paper of [Taylor and Letham, 2017](https://peerj.com/preprints/3190/).

> Questions:
> Contact Guillaume Azarias at guillaume.azarias@hotmail.com

## Import the relevant library

In [2]:
import os
import pickle
import numpy as np
import seaborn as sns
sns.set()
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

import datetime
from datetime import timedelta
import qgrid

# Note that the interactive plot may not work in Jupyter lab, but only in Jupyter Notebook (conflict of javascripts)
%matplotlib widget 

In [3]:
import fbprophet
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics
from fbprophet.plot import plot_cross_validation_metric

In [4]:
fbprophet.__version__

'0.6'

In [5]:
from sklearn.model_selection import ParameterGrid
import itertools
from random import sample

In [6]:
# Import the functions from the helper.py
from helper import load_ds, df_dev_formater, find_index, df_generator, prophet_fit, prophet_plot, get_outliers, prophet, GridSearch_Prophet

## Read the WESAD data

The dimensions of the dataset depend on both the device and parameters:

|     Device     | Location|Parameter|Acq. frequency|Number of dimensions|Data points (S5)| Duration (S5)|
|:---------------|:-------:|:-------:|:------------:|:------------------:|:--------------:|:------------:|
|**RespiBAN Pro**|chest    | ACC     |700Hz         |**3**               |4496100         |6'423sec      |
|                |         | ECG     |"             |1                   |                |              |
|                |         | EDA     |"             |1                   |                |              |
|                |         | EMG     |"             |1                   |                |              |
|                |         | RESP    |"             |1                   |                |              |
|                |         | TEMP    |"             |1                   |                |              |
|                |         |         |              |                    |                |              |
|**Empatica E4** |wrist    | ACC     |32Hz          |**3**               |200256          |6'258sec      |
|                |         | BVP     |64Hz          |1                   |400512          |              |
|                |         | EDA     |4Hz           |1                   |25032           |              |
|                |         | TEMP    |4Hz           |1                   |25032           |              |

*Note that ACC is a matrix of 3 dimensions for the 3 spatial dimensions.*

*'ECG', 'EDA', 'EMG', 'Resp', 'Temp' have each 1 dimension.*

In [7]:
freq = np.array([4, 700, 700, 700, 700, 700, 700, 32, 64, 4, 4, 700])
freq_df = pd.Series(freq, index= ['working_freq', 'ACC_chest', 'ECG_chest', 'EDA_chest', 'EMG_chest', 'Resp_chest', 'Temp_chest', 'ACC_wrist', 'BVP_wrist', 'EDA_wrist', 'TEMP_wrist', 'label'])
freq_df

working_freq      4
ACC_chest       700
ECG_chest       700
EDA_chest       700
EMG_chest       700
Resp_chest      700
Temp_chest      700
ACC_wrist        32
BVP_wrist        64
EDA_wrist         4
TEMP_wrist        4
label           700
dtype: int64

In [8]:
# Define the working frequency, eg the frequency to adjust all data
working_freq = str(int(1000/freq_df.loc['working_freq'])) + 'L'
working_freq

'250L'

*Note:* The class read_data_of_one_subject was originally written by [aganjag](https://github.com/jaganjag/stress_affect_detection/blob/master/prototype.ipynb).

In [9]:
# obj_data[subject] = read_data_one_subject(data_set_path, subject)
class read_data_of_one_subject:
    """Read data from WESAD dataset"""
    def __init__(self, path, subject):
        self.keys = ['label', 'subject', 'signal']
        self.signal_keys = ['wrist', 'chest']
        self.chest_sensor_keys = ['ACC', 'ECG', 'EDA', 'EMG', 'Resp', 'Temp']
        self.wrist_sensor_keys = ['ACC', 'BVP', 'EDA', 'TEMP']
        os.chdir(path) # Change the current working directory to path
        os.chdir(subject) # Change the current working directory to path. Why not using data_set_path ?
        with open(subject + '.pkl', 'rb') as file: # with will automatically close the file after the nested block of code
            data = pickle.load(file, encoding='latin1')
        self.data = data

    def get_labels(self):
        return self.data[self.keys[0]]

    def get_wrist_data(self):
        """"""
        #label = self.data[self.keys[0]]
        assert subject == self.data[self.keys[1]], 'WARNING: Mixing up the data from different persons'
        signal = self.data[self.keys[2]]
        wrist_data = signal[self.signal_keys[0]]
        #wrist_ACC = wrist_data[self.wrist_sensor_keys[0]]
        #wrist_ECG = wrist_data[self.wrist_sensor_keys[1]]
        return wrist_data

    def get_chest_data(self):
        """"""
        assert subject == self.data[self.keys[1]], 'WARNING: Mixing up the data from different persons'
        signal = self.data[self.keys[2]]
        chest_data = signal[self.signal_keys[1]]
        return chest_data

In [10]:
data_set_path = "../../Data/WESAD"
subject = 'S5'

In [11]:
obj_data = {}
obj_data[subject] = read_data_of_one_subject(data_set_path, subject)

*Workplan:*

**A) Exploratory data analysis**

    1) Discard for now the ACC data. Preliminary results on other parameters may guide the ways to investigate the accelerometer data
    2) Get the study protocol
    3) Use rolling.mean() to synchronise the data at the same frequency
    4) Synchronise data
    5) Include label data if possible
    6) Plot data
    7) Segmentation per task
    8) Quality control 

**B) Perform time series forecasting**

    1) ADCF test
    2) Prophet
    3) ARIMA

## Get the study protocol
*From the wesad_readme.pdf:*

The order of the different conditions is defined on the second line in SX_quest.csv. Please refer to [1] for further details on each of the conditions (see Section 3.3 there). Please ignore the elements “bRead”, “fRead”, and “sRead”: these are not relevant for this dataset.
The time interval of each condition is defined as start and end time, see the lines 3 and 4 in SX_quest.csv. Time is given in the format [minutes.seconds]. Time is counted from the start of the RespiBAN device’s start of recording.

### Study protocol from the _quest.csv file

In [12]:
print(os.getcwd())

/Users/guillaume/Documents/Projects/Data/WESAD/S5


In [13]:
SX_quest_filename = os.getcwd() + '/' + subject + '_quest.csv'
print(SX_quest_filename)
# bp_data = pd.read_csv("/Users/guillaume/Documents/Projects/Data/WESAD/S2/S2_quest.csv", header=1, delimiter=';')
study_protocol_raw = pd.read_csv(SX_quest_filename, delimiter=';')
# study_protocol_raw.head()

/Users/guillaume/Documents/Projects/Data/WESAD/S5/S5_quest.csv


In [14]:
# Create a table with the interval of every steps
study_protocol = study_protocol_raw.iloc[1:3, 1:6]
study_protocol = study_protocol.transpose().astype(float)
study_protocol.columns = ['start', 'end']
study_protocol['task'] = study_protocol_raw.iloc[0, 1:6].transpose()
study_protocol = study_protocol.reset_index(drop=True)
study_protocol
# study_protocol.dtypes

,start,end,task
0,5.37,25.55,Base
1,32.00,38.34,Fun
2,45.43,52.40,Medi 1
3,61.00,72.05,TSST
4,92.15,99.12,Medi 2


In [15]:
# Create a dataframe with the time formatted as datetime
# Note that the frequency chosen was 4Hz to match the lowest frequency of acquisition (250ms)
total_duration = study_protocol.end.max()
data = pd.DataFrame()
begin_df = datetime.datetime(2020, 1, 1) # For reading convenience
end_df = begin_df + timedelta(minutes=int(total_duration)) + timedelta(seconds=total_duration-int(total_duration))
data['time'] = pd.date_range(start=begin_df, end=end_df, freq=working_freq).to_pydatetime().tolist()
data['task'] = np.nan
# data

In [16]:
# Annotate the task in the data['task']
for row in range(study_protocol.shape[0]):
    # Datetime index of the beginning of the task
    begin_state = study_protocol.iloc[row, 0]
    begin = begin_df + timedelta(minutes=int(begin_state)) + timedelta(seconds=begin_state-int(begin_state))

    # Datetime index of the end of the task
    end_state = study_protocol.iloc[row, 1]
    end = begin_df + timedelta(minutes=int(end_state)) + timedelta(seconds=end_state-int(end_state))

    # Fill the task column according to the begin and end of task
    data.loc[(data['time'] >= begin) & (data['time'] <= end), 'task'] = study_protocol.iloc[row, 2]

# Show data
qgrid_widget = qgrid.show_grid(data, show_toolbar=True)
qgrid.show_grid(data)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Graphical representation of the study protocol

In [17]:
# Attribute an arbitrary value to a task for graphical display of the study protocol
data_graph = data
data_graph['arbitrary_index'] = np.zeros

data_graph.loc[data_graph['task'] == 'Base', 'arbitrary_index'] = 1
data_graph.loc[data_graph['task'] == 'Fun', 'arbitrary_index'] = 3
data_graph.loc[data_graph['task'] == 'Medi 1', 'arbitrary_index'] = 4
data_graph.loc[data_graph['task'] == 'TSST', 'arbitrary_index'] = 2
data_graph.loc[data_graph['task'] == 'Medi 2', 'arbitrary_index'] = 4

data_graph['arbitrary_index'] = pd.to_numeric(data_graph['arbitrary_index'], errors='coerce')

# # Show data
# qgrid_widget = qgrid.show_grid(data_graph, show_toolbar=True)
# qgrid.show_grid(data_graph)

In [18]:
# Plot
fig_sp, ax = plt.subplots(figsize=(8, 4))

plt.plot('time', 'arbitrary_index', data=data_graph, color='darkblue', marker='o',linestyle='dashed', linewidth=0.5, markersize=2)
plt.gcf().autofmt_xdate()
myFmt = DateFormatter("%H:%M")
ax.xaxis.set_major_formatter(myFmt)
plt.xlabel('Time elapsed (hh:mm)', fontsize=15)
plt.ylim(0,6)
plt.ylabel('Arbitrary index', fontsize=15)
name = 'Study protocol for the subject ' + subject
plt.title(name, fontsize=20)

# Graph annotation
for row in range(study_protocol.shape[0]):
    # Datetime index of the beginning of the task
    begin_state = study_protocol.iloc[row, 0]
    begin = begin_df + timedelta(minutes=int(begin_state)) + timedelta(seconds=begin_state-int(begin_state))

    # Datetime index of the end of the task
    end_state = study_protocol.iloc[row, 1]
    end = begin_df + timedelta(minutes=int(end_state)) + timedelta(seconds=end_state-int(end_state))

    # Draw a rectangle and annotate the graph
    ax.axvspan(begin, end, facecolor='b', alpha=0.2)
    text_location = begin+((end-begin)/2)*1/2
    ax.annotate(study_protocol.iloc[row, 2], xy=(begin, 5), xytext=(text_location, 5.5), fontsize=10)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Get the results of the self-assesment

In [19]:
study_protocol_raw
# Show data
# qgrid_widget = qgrid.show_grid(study_protocol_raw, show_toolbar=True)
# qgrid.show_grid(study_protocol_raw)

,# Subj,S5,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,# ORDER,Base,Fun,Medi 1,TSST,Medi 2,bRead,fRead,sRead,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,# START,5.37,32,45.43,61,92.15,26.15,41.15,75.28,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,# END,25.55,38.34,52.4,72.05,99.12,27.47,42.45,76.32,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,# PANAS,1,1,4,3,1,3,1,1,1.0,...,4.0,4.0,1.0,1.0,2.0,1.0,3.0,1.0,NaN,NaN
5,# PANAS,1,1,3,3,1,2,1,1,1.0,...,1.0,3.0,1.0,1.0,2.0,1.0,2.0,1.0,NaN,NaN
6,# PANAS,1,1,3,1,1,3,1,1,1.0,...,3.0,3.0,1.0,1.0,1.0,1.0,4.0,1.0,NaN,NaN
7,# PANAS,5,1,4,2,2,3,1,2,1.0,...,3.0,4.0,3.0,1.0,4.0,1.0,2.0,1.0,2.0,1.0
8,# PANAS,1,1,3,2,1,2,1,1,1.0,...,2.0,2.0,1.0,1.0,1.0,1.0,4.0,1.0,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**The order of result of the self-assessment is listed below:**
- line 0: Condition
- lines 1-2: Start and end of the condition
- line 3: *NaN line for data separation*
- lines 4-8: PANAS result with the 26 different feeling in columns (columns 1-27), and scores (1 = Not at all, 2 = A little bit, 3 = Somewhat, 4 = Very much, 5 = Extremely) for the conditions: Base (line 4) Fun (line 5) Medi 1 (line 6) TSST (line 7) and Medi 2 (line 8). *Note that there are 2 more features for the Stress condition only.*
- line 9: *NaN line for data separation*
- lines 10-14: STAI result with the 6 different feelings in columns and scores (1 = Not at all, 2 = Somewhat, 3 = Moderately so, 4 = Very much so) for the conditions: Base (line 10) Fun (line 11) Medi 1 (line 12) TSST (line 13) and Medi 2 (line 14). 
- line 15: *NaN line for data separation*
- lines 16-20: SAM (Self-Assessment Manikins) results with the 2 different feelings (valence and arousal) in columns  for the conditions: Base (line 16) Fun (line 17) Medi 1 (line 18) TSST (line 19) and Medi 2 (line 20).
- line 21: *NaN line for data separation*
- lines 22: SSSQ result with the 6 different feeling and scores (1 = Not at all, 2 = A little bit, 3 = Somewhat, 4 = Very much, 5 = Extremely) for the stress condition only.

*TO DO*: pool, transpose and normalise data. Verify SSSQ information

In [1]:
# self_assessment = pd.DataFrame()
# self_assessment = study_protocol_raw.iloc[0, 1:5].T

# # Base results
# self_assessment.iloc[0, 1:27] = study_protocol_raw.iloc[4, 1:27]
# self_assessment

## Get the wrist data and adjust to the working frequency (4Hz)

|     Device     | Location|Parameter|Acq. frequency|Number of dimensions|Data points (S5)| Duration (S5)|
|:---------------|:-------:|:-------:|:------------:|:------------------:|:--------------:|:------------:|
|**Empatica E4** |wrist    | ACC     |32Hz          |**3**               |200256          |6'258sec      |
|                |         | BVP     |64Hz          |1                   |400512          |              |
|                |         | EDA     |4Hz           |1                   |25032           |              |
|                |         | TEMP    |4Hz           |1                   |25032           |              |

In [19]:
freq_df

working_freq      4
ACC_chest       700
ECG_chest       700
EDA_chest       700
EMG_chest       700
Resp_chest      700
Temp_chest      700
ACC_wrist        32
BVP_wrist        64
EDA_wrist         4
TEMP_wrist        4
label           700
dtype: int64

In [20]:
wrist_data_dict = obj_data[subject].get_wrist_data()

In [21]:
# Extraction of numbers of data
wrist_dict_length = {key: len(value) for key, value in wrist_data_dict.items()}
print('Original numbers of data per parameter: ' + str(wrist_dict_length))
wrist_ser_length = pd.Series(wrist_dict_length)
df_wrist = pd.DataFrame()

# Adjust all data to the same frequence
for wrist_param, param_length in wrist_ser_length.items():
    # Generate the frequence in microseconds (U) from the acquisition frequency
    freq = str(int(1000000/freq_df.loc[wrist_param + '_wrist'])) + 'U'
    # Generate temporary dataset 
    index = pd.date_range(start='1/1/2020', periods=param_length, freq=freq)
    df_temp_raw = pd.DataFrame(wrist_data_dict[wrist_param], index=index)
    if wrist_param == 'ACC':
        df_temp_raw.columns = ['ACC_wrist_x', 'ACC_wrist_y', 'ACC_wrist_z']
    else:
        df_temp_raw.columns = [wrist_param + '_wrist']
    # Resampling
    df_temp = df_temp_raw.resample(working_freq).pad()
    # Append the wrist data
    if df_wrist.shape[1]==0:
        df_wrist = df_temp
    else:
        df_wrist = pd.concat([df_wrist, df_temp], axis=1)

print('Resampled data adjusted to ' + str(freq_df['working_freq']) + 'Hz in the pandas DataFrame df_wrist:')
df_wrist

Original numbers of data per parameter: {'ACC': 200256, 'BVP': 400512, 'EDA': 25032, 'TEMP': 25032}
Resampled data adjusted to 4Hz in the pandas DataFrame df_wrist:


,ACC_wrist_x,ACC_wrist_y,ACC_wrist_z,BVP_wrist,EDA_wrist,TEMP_wrist
2020-01-01 00:00:00.000,-16.0,-45.0,127.0,-7.25,0.547723,34.09
2020-01-01 00:00:00.250,60.0,45.0,-1.0,-8.24,0.481218,34.09
2020-01-01 00:00:00.500,45.0,41.0,-32.0,-11.14,0.518239,34.09
2020-01-01 00:00:00.750,72.0,33.0,7.0,-32.96,0.440223,34.11
2020-01-01 00:00:01.000,51.0,39.0,9.0,79.55,0.391623,34.11
...,...,...,...,...,...,...
2020-01-01 01:44:16.750,45.0,39.0,21.0,-26.64,1.004306,31.03
2020-01-01 01:44:17.000,45.0,39.0,21.0,-0.84,0.997912,31.03
2020-01-01 01:44:17.250,46.0,38.0,21.0,-22.00,0.985122,31.03
2020-01-01 01:44:17.500,45.0,39.0,20.0,-2.90,0.965938,31.03


## Chest data from the .pkl file adjusted to 4Hz

|     Device     | Location|Parameter|Acq. frequency|Number of dimensions|Data points (S5)| Duration (S5)|
|:---------------|:-------:|:-------:|:------------:|:------------------:|:--------------:|:------------:|
|**RespiBAN Pro**|chest    | ACC     |700Hz         |**3**               |4496100         |6'423sec      |
|                |         | ECG     |"             |1                   |                |              |
|                |         | EDA     |"             |1                   |                |              |
|                |         | EMG     |"             |1                   |                |              |
|                |         | RESP    |"             |1                   |                |              |
|                |         | TEMP    |"             |1                   |                |              |

In [27]:
chest_data_dict = obj_data[subject].get_chest_data()

In [28]:
# Extraction of numbers of data
chest_dict_length = {key: len(value) for key, value in chest_data_dict.items()}
print('Original numbers of data per parameter: ' + str(chest_dict_length))
chest_ser_length = pd.Series(chest_dict_length)
df_chest = pd.DataFrame()

# Adjust all data to the same frequence
for chest_param, param_length in chest_ser_length.items():
    # Generate the frequence in microseconds (U) from the acquisition frequency
    freq = str(int(1000000/freq_df.loc[chest_param + '_chest'])) + 'U'
    # Generate temporary dataset 
    index = pd.date_range(start='1/1/2020', periods=param_length, freq=freq)
    df_temp_raw = pd.DataFrame(chest_data_dict[chest_param], index=index)
    if chest_param == 'ACC':
        df_temp_raw.columns = ['ACC_chest_x', 'ACC_chest_y', 'ACC_chest_z']
    else:
        df_temp_raw.columns = [chest_param + '_chest']
    # Resampling
    df_temp = df_temp_raw.resample(working_freq).pad()
    # Append the chest data
    if df_chest.shape[1]==0:
        df_chest = df_temp
    else:
        df_chest = pd.concat([df_chest, df_temp], axis=1)

print('Resampled data adjusted to ' + str(freq_df['working_freq']) + 'Hz in the pandas DataFrame df_chest:')
df_chest

Original numbers of data per parameter: {'ACC': 4380600, 'ECG': 4380600, 'EMG': 4380600, 'EDA': 4380600, 'Temp': 4380600, 'Resp': 4380600}
Resampled data adjusted to 4Hz in the pandas DataFrame df_chest:


,ACC_chest_x,ACC_chest_y,ACC_chest_z,ECG_chest,EMG_chest,EDA_chest,Temp_chest,Resp_chest
2020-01-01 00:00:00.000,0.8606,0.0742,0.8570,-0.275803,0.016800,3.888321,34.119934,0.044250
2020-01-01 00:00:00.250,0.9122,-0.0464,-0.1102,0.125427,-0.001144,3.870010,34.130615,-0.456238
2020-01-01 00:00:00.500,0.7784,0.1914,-0.1982,-0.018127,-0.004807,3.883743,34.130615,-0.331116
2020-01-01 00:00:00.750,0.9096,-0.0700,-0.1038,0.064774,-0.004211,3.874588,34.129089,0.450134
2020-01-01 00:00:01.000,0.9066,-0.0538,-0.0962,-0.045090,-0.012222,3.880310,34.086426,1.849365
...,...,...,...,...,...,...,...,...
2020-01-01 01:44:14.250,0.9118,-0.0464,-0.1032,-0.006271,-0.010391,10.274506,34.977234,-3.100586
2020-01-01 01:44:14.500,0.9102,-0.0530,-0.1064,-0.072647,0.001419,10.261917,34.877014,-2.728271
2020-01-01 01:44:14.750,0.9016,-0.0486,-0.1166,0.013367,-0.006042,10.252380,34.949463,-0.717163
2020-01-01 01:44:15.000,0.9054,-0.0590,-0.1270,0.000504,-0.014648,10.234451,34.978790,1.863098


## Get the label data

‘label’: ID of the respective study protocol condition, sampled at 700 Hz.
The following IDs are provided:
    - 0 = not defined / transient
    - 1 = baseline
    - 2 = stress
    - 3 = amusement
    - 4 = meditation
    - 5/6/7 = should be ignored in this dataset

In [29]:
labels = {}
labels[subject] = obj_data[subject].get_labels()
labels_dict = obj_data[subject].get_labels()

In [33]:
freq = str(int(1000000/freq_df.loc['label'])) + 'U' # U means microseconds
index = pd.date_range(start='1/1/2020', periods=len(labels_dict), freq=freq)
df_label = pd.DataFrame(labels_dict, index=index)
df_label = df_label.resample(working_freq).pad()
df_label['time'] = df_label.index
df_label.columns = ['label', 'time']
# Ignore 5/6/7
df_label.loc[df_label['label'] > 4, 'label'] = 0
df_label

,label,time
2020-01-01 00:00:00.000,0,2020-01-01 00:00:00.000
2020-01-01 00:00:00.250,0,2020-01-01 00:00:00.250
2020-01-01 00:00:00.500,0,2020-01-01 00:00:00.500
2020-01-01 00:00:00.750,0,2020-01-01 00:00:00.750
2020-01-01 00:00:01.000,0,2020-01-01 00:00:01.000
...,...,...
2020-01-01 01:44:14.250,0,2020-01-01 01:44:14.250
2020-01-01 01:44:14.500,0,2020-01-01 01:44:14.500
2020-01-01 01:44:14.750,0,2020-01-01 01:44:14.750
2020-01-01 01:44:15.000,0,2020-01-01 01:44:15.000


In [34]:
# Plot
fig_sp, ax = plt.subplots(figsize=(8, 4))

plt.plot('time', 'label', data=df_label, color='darkblue', marker='o',linestyle='dashed', linewidth=0.5, markersize=2)
plt.gcf().autofmt_xdate()
myFmt = DateFormatter("%H:%M")
ax.xaxis.set_major_formatter(myFmt)
plt.xlabel('Time elapsed (hh:mm)', fontsize=15)
plt.ylim(0,6)
plt.ylabel('Label', fontsize=15)
name = 'Label data for the subject ' + subject
plt.title(name, fontsize=20)

# Graph annotation
for row in range(study_protocol.shape[0]):
    # Datetime index of the beginning of the task
    begin_state = study_protocol.iloc[row, 0]
    begin = begin_df + timedelta(minutes=int(begin_state)) + timedelta(seconds=begin_state-int(begin_state))

    # Datetime index of the end of the task
    end_state = study_protocol.iloc[row, 1]
    end = begin_df + timedelta(minutes=int(end_state)) + timedelta(seconds=end_state-int(end_state))

    # Draw a rectangle and annotate the graph
    ax.axvspan(begin, end, facecolor='b', alpha=0.2)
    text_location = begin+((end-begin)/2)*1/2
    ax.annotate(study_protocol.iloc[row, 2], xy=(begin, 5), xytext=(text_location, 5.5), fontsize=10)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# There is a lag between the data extracted from the _quest.csv file and the synchronized data:
    - Check if there is any synchronisation data in the pkl file
    - Look how to properly merge the datasets that don't have the same dimensions !